In [1]:
import torch

In [37]:
pred = torch.tensor([[0.81, 0.8, 0.7, 0.6, 0.5]])
truth = torch.tensor([[5, 4, 3, 2, 1]], dtype=torch.float32)-1

In [38]:
pred.argsort(dim=1).float()

tensor([[4., 3., 2., 1., 0.]])

In [39]:
loss = torch.nn.functional.cross_entropy(pred.argsort(dim=1).float().softmax(dim=1), truth.softmax(dim=1))
loss

tensor(1.3702)

In [40]:
pred.argsort(dim=1).float().softmax(dim=1), truth.softmax(dim=1)

(tensor([[0.6364, 0.2341, 0.0861, 0.0317, 0.0117]]),
 tensor([[0.6364, 0.2341, 0.0861, 0.0317, 0.0117]]))

In [41]:
# ListNet loss introduced in "Learning to Rank: From Pairwise Approach to Listwise Approach".
preds_smax = pred.argsort(dim=1).float().softmax(dim=1)# + eps
preds_log = torch.log(preds_smax)

torch.mean(-torch.sum(truth.softmax(dim=1) * preds_log, dim=1))

tensor(1.0000)

In [1]:
from common import load_all_csv, load_x264

In [2]:
df1 = load_all_csv("../data/res_ugc/")

In [3]:
df1.columns

AttributeError: 'tuple' object has no attribute 'columns'

In [ ]:
df = df1[0].dropna()
X = df.loc[:, df1[0].columns[1:-8]]
y = df.loc[:, 'kbs']

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, SplineTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_predict

df_preproc = ColumnTransformer(
    [
        (
            "cat",
            OneHotEncoder(max_categories=10, handle_unknown="infrequent_if_exist"),
            make_column_selector(dtype_include="category"),
        ),
        (
            "num",
            SplineTransformer(n_knots=10),
            make_column_selector(dtype_include="number"),
        ),
    ],
)
mlpr = make_pipeline(
    df_preproc,
    MLPRegressor(
        hidden_layer_sizes=[256]*3,
    ))
mlp_scores = cross_val_predict(mlpr, X, y, cv=4)

/Users/helge/src/resist/RESIST_perf_clustering/.venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


,cabac,ref,deblock,analyse,me,subme,mixed_ref,me_range,trellis,8x8dct,fast_pskip,chroma_qp_offset,bframes,b_pyramid,b_adapt,direct,weightb,open_gop,weightp,scenecut,rc_lookahead,mbtree,qpmax,aq-mode
0,0,1,0:0:0,0:0,dia,0,0,16,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,0,69,0
1,1,2,1:0:0,0x3:0x113,hex,6,1,16,1,1,1,0,8,2.0,1.0,spatial,1.0,0.0,1,0.0,60.0,1,69,1
2,1,2,1:0:0,0x3:0x113,hex,6,1,16,1,1,1,0,8,2.0,1.0,auto,1.0,0.0,1,0.0,10.0,1,69,0
3,1,2,0:0:0,0x3:0x3,umh,6,1,16,1,1,1,0,8,2.0,1.0,spatial,1.0,0.0,1,0.0,10.0,1,69,0
4,1,16,1:0:0,0x3:0x113,hex,6,1,24,1,1,1,-2,3,2.0,1.0,auto,1.0,0.0,1,40.0,30.0,0,69,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1,2,1:0:0,0x3:0x113,hex,4,1,16,0,1,1,0,3,2.0,2.0,auto,1.0,0.0,1,40.0,20.0,1,69,1
197,1,2,0:0:0,0x3:0x113,hex,4,0,24,0,1,1,0,3,2.0,1.0,auto,1.0,0.0,1,40.0,30.0,1,69,1
198,1,5,1:0:0,0x3:0x113,hex,6,1,16,1,1,0,-2,3,NaN,1.0,spatial,1.0,0.0,1,40.0,30.0,1,69,1
199,0,3,0:0:0,0x113:0x113,hex,6,1,16,1,1,1,-2,3,2.0,1.0,auto,1.0,0.0,2,40.0,30.0,0,69,1
